In [2]:
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

df = pd.read_feather("cache.feather") #[['round_trip_hrs_mean','_adi_decile','population']]

def _buckets(x):
    if x < 6:
        return 6
    elif x < 12:
        return 12
    elif x < 18:
        return 18
    elif x < 24:
        return 24
    else: 
        return 30
    
df['_time_bucket'] = df['round_trip_hrs_mean'].apply(_buckets)
# df['_total_loss'] = df['population'] * df['_hrs_bucket']
df


,origin_state,zip5,round_trip_hrs_mean,_adi_decile,population,_time_bucket
0,AL,36608,18.710000,6.0,36864.0,24
1,AL,36350,18.173333,8.0,6540.0,24
2,AL,36401,17.070000,8.0,8314.0,18
3,AL,36069,15.606667,7.0,1935.0,18
4,AL,36904,15.520000,8.0,4098.0,18
...,...,...,...,...,...,...
593,WY,83001,15.386667,0.0,15851.0,18
594,WY,82701,12.080000,4.0,5455.0,18
595,WY,82701,12.080000,6.0,5455.0,18
596,WY,82411,15.533333,4.0,664.0,18


In [3]:
dfg = df.groupby(['_adi_decile','round_trip_hrs_mean']).agg(population=("population","sum")).reset_index()
scaler = MinMaxScaler()
dfg[['_population']] = scaler.fit_transform(dfg[['population']])
dfg['_loss'] = dfg['round_trip_hrs_mean'] * dfg['_population']
display(dfg.head())
px.scatter(dfg, x='_adi_decile', y='_loss')

,_adi_decile,round_trip_hrs_mean,population,_population,_loss
0,0.0,1.260000,21850.0,0.292089,0.368032
1,0.0,1.393333,55990.0,0.748909,1.043481
2,0.0,1.600000,9416.0,0.125713,0.201140
3,0.0,2.150000,11315.0,0.151123,0.324914
4,0.0,6.230000,3020.0,0.040129,0.250004


In [4]:
dfg = df.groupby(['_adi_decile','_time_bucket']).agg(population=("population","sum")).reset_index()
scaler = MinMaxScaler()
dfg[['_population']] = scaler.fit_transform(dfg[['population']])
dfg.head()

,_adi_decile,_time_bucket,population,_population
0,0.0,6,98571.0,0.179083
1,0.0,12,27077.0,0.048035
2,0.0,18,44821.0,0.080560
3,1.0,6,111542.0,0.202859
4,1.0,18,70018.0,0.126746


In [6]:
dfg['_loss'] = dfg['_time_bucket'] * dfg['_population']

In [7]:
# px.scatter(dfg, x='_adi_decile', y='_loss', trendline='ols')

In [ ]:
# px.scatter(dfg, x='')

In [ ]:
# scaler = MinMaxScaler()
# df[df.columns] = scaler.fit_transform(df[df.columns])
# df

In [3]:
from post_roe.query import TinyQuery as tq
from post_roe.google_helpers import load_cache

adi = tq.adi_with_status_query()
closest_clinics = load_cache()
distance_from_origin_stats = closest_clinics.groupby(['origin_state','origin_lat','origin_lng','zip5']).agg(
        k=("origin_lat","count"),
        geodesic_mean=("geodesic_miles","mean"), 
        drive_miles_mean=("drive_miles","mean"),
        round_trip_hrs_mean=("_roundtrip_hours","mean")
    ).reset_index()
distance_from_origin_stats = distance_from_origin_stats.merge(adi[['zip5','_adi_decile','population']], how='left')
df = distance_from_origin_stats[['origin_state','zip5','round_trip_hrs_mean','_adi_decile','population']].reset_index(drop=True)
df.to_feather("cache.feather")

NameError: name 'distance_from_origin_stats' is not defined

# _wip

In [38]:
df = distance_from_origin_stats
df = df[['adi_median','population','round_trip_drive_hrs_mean']].reset_index(drop=True)
df['_loss'] = (df['round_trip_drive_hrs_mean'] * df['population'] / 1000).dropna().astype(int)
df

,adi_median,population,round_trip_drive_hrs_mean,_loss
0,69.0,36864.0,18.710000,689.0
1,87.0,6540.0,18.173333,118.0
2,89.0,8314.0,17.070000,141.0
3,77.0,1935.0,15.606667,30.0
4,89.0,4098.0,15.520000,63.0
...,...,...,...,...
593,6.0,15851.0,15.386667,243.0
594,42.0,5455.0,12.080000,65.0
595,63.0,5455.0,12.080000,65.0
596,47.0,664.0,15.533333,10.0


In [40]:
px.scatter(
    df, 
    x='adi_median',
    y='_loss',
    trendline='ols'
)

In [5]:
# px.scatter(df, x='drive_duration', y='adi_median', trendline='ols', hover_data=['zip5'])
px.scatter(df.dropna(subset=['population']), x='drive_duration', y='adi_median', size='population', hover_data=['zip5'])

In [ ]:
# i = states.index[states['state'] == "GA"][0]
# states.at[i, '_status'] = "protected"
# states
